In [ ]:
import sys
print(sys.version)
print(*sys.path, sep='\n')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision as tv

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

num_workers = os.cpu_count()
print(num_workers)

cuda:0
6


In [4]:
# parameters for trained CNN
root = os.path.join('.', 'dataset_root')
# mean, std = [0.13066046], [0.30150425] # based on training set
batch_size = 128
lr = 0.01
momentum=0.9
step_size=6
gamma=0.1
epochs = 40

In [5]:
def imshow(tensor_grid, mean=0., std=1., title=None):
    assert isinstance(tensor_grid, torch.Tensor)
    assert len(tensor_grid.size()) == 4, \
        f'For a batch of images only, {tensor_grid.size()} '
    
    tensor_grid = tv.utils.make_grid(tensor_grid)
    grid = tensor_grid.numpy().transpose((1,2,0))
    grid = std * grid + mean
    grid = np.clip(grid, 0, 1)
    plt.imshow(grid)
    
    if title is not None:
        plt.title(title)
        
    plt.pause(0.001)

In [6]:
# prepare data
# foolbox model expects raw numpy array as image
transform = tv.transforms.Compose([
        tv.transforms.ToTensor(),
#         tv.transforms.Normalize(mean, std)
])
train_dataset = tv.datasets.MNIST(
    root,
    train=True, 
    download=True,
    transform=transform)
test_dataset = tv.datasets.MNIST(
    root,
    train=False, 
    download=True,
    transform=transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)

In [7]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.dropout = nn.Dropout2d(0.2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(800, 128)
        self.fc2 = nn.Linear(128, 84)
        self.fc3 = nn.Linear(84, 10)
                             
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
        return x

In [8]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.fc2 = nn.Linear(128, 84)
        self.fc3 = nn.Linear(84, 10)
                             
    def forward(self, x):
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
def train(model, loader, optimizer):
    model.train()
    total_loss = 0.
    corrects = 0
        
    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        batch_size = x.size(0)
        
        optimizer.zero_grad()
        output = model(x)
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()

        # for display
        total_loss += loss.item() * batch_size
        preds = output.max(1, keepdim=True)[1]
        corrects += preds.eq(y.view_as(preds)).sum().item()
    
    n = len(train_loader.dataset)
    total_loss = total_loss / n
    accuracy = corrects / n
    return total_loss, accuracy

In [10]:
def validate(model, loader):
    model.eval()
    total_loss = 0.
    corrects = 0
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            batch_size = x.size(0)
            output = model(x)
            loss = F.nll_loss(output, y)
            total_loss += loss.item() * batch_size
            preds = output.max(1, keepdim=True)[1]
            corrects += preds.eq(y.view_as(preds)).sum().item()
    
    n = len(test_loader.dataset)
    total_loss = total_loss / n
    accuracy = corrects / n
    return total_loss, accuracy

In [11]:
# NOTE: NO GPU AT SCHOOL!
model1 = Net1()
model2 = Net2()
model_softmax = torch.nn.Sequential(
    model1,
    model2,
    torch.nn.LogSoftmax(dim=1)
)
model_softmax.to(device)
optimizer = torch.optim.SGD(model_softmax.parameters(), lr=lr, momentum=momentum)
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, 
    step_size=step_size, 
    gamma=gamma)

In [14]:
since = time.time()
for epoch in range(epochs):
    start = time.time()
    
    best_model_state = copy.deepcopy(model_softmax.state_dict())
    best_tr_acc = 0.0
    best_va_acc = 0.0
    prev_loss = 1e10
    
    tr_loss, tr_acc = train(model_softmax, train_loader, optimizer)
    va_loss, va_acc = validate(model_softmax, test_loader)
    lr_scheduler.step()
    
    # save best result
    if tr_acc >= best_tr_acc and va_acc >= best_va_acc:
        best_model_state = copy.deepcopy(model_softmax.state_dict())
        best_tr_acc = tr_acc
        best_va_acc = va_acc
    
    # display
    time_elapsed = time.time() - start
    print(('[{:2d}] {:.0f}m {:.1f}s Train Loss: {:.4f} Accuracy: {:.4f}%, ' +
        'Test Loss: {:.4f} Accuracy: {:.4f}%').format(
            epoch+1, time_elapsed // 60, time_elapsed % 60,
            tr_loss, tr_acc*100.,
            va_loss, va_acc*100.))
    
time_elapsed = time.time() - since
print('Training completed in {:.0f}m {:.1f}s'.format(
    time_elapsed // 60,
    time_elapsed % 60))
print(f'Best val Acc: {best_va_acc:4f}')

[ 1] 0m 1.8s Train Loss: 0.1634 Accuracy: 94.9983%, Test Loss: 0.0839 Accuracy: 97.3000%
[ 2] 0m 1.7s Train Loss: 0.1004 Accuracy: 96.8533%, Test Loss: 0.0596 Accuracy: 98.0900%
[ 3] 0m 1.8s Train Loss: 0.0764 Accuracy: 97.6200%, Test Loss: 0.0636 Accuracy: 97.9600%
[ 4] 0m 1.7s Train Loss: 0.0658 Accuracy: 97.9450%, Test Loss: 0.0434 Accuracy: 98.4400%
[ 5] 0m 1.7s Train Loss: 0.0557 Accuracy: 98.2983%, Test Loss: 0.0403 Accuracy: 98.6200%
[ 6] 0m 1.8s Train Loss: 0.0413 Accuracy: 98.7300%, Test Loss: 0.0342 Accuracy: 98.8500%
[ 7] 0m 1.7s Train Loss: 0.0382 Accuracy: 98.7917%, Test Loss: 0.0337 Accuracy: 98.8600%
[ 8] 0m 1.7s Train Loss: 0.0373 Accuracy: 98.8233%, Test Loss: 0.0330 Accuracy: 98.8700%
[ 9] 0m 1.7s Train Loss: 0.0363 Accuracy: 98.8717%, Test Loss: 0.0325 Accuracy: 98.9500%
[10] 0m 1.7s Train Loss: 0.0364 Accuracy: 98.8800%, Test Loss: 0.0335 Accuracy: 98.8300%
[11] 0m 1.7s Train Loss: 0.0350 Accuracy: 98.9333%, Test Loss: 0.0334 Accuracy: 98.8400%
[12] 0m 1.7s Train Lo

In [15]:
torch.save(model_softmax.state_dict(), 'mnist_model3.pt')